- 새로운 가상환경 생성
conda create --name codyssey pandas
- 가상 환경 실행
conda activate codyssey
- 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악

In [1]:
import pandas as pd

In [2]:
# 데이터 읽기
data = pd.read_csv('prob-0101.csv')

In [3]:
# 각 항목 계산
num_movies = len(data)  # 영화 개수
num_distributors = data['배급사'].nunique()  # 배급사 수 (중복 제거)
num_directors = data['감독'].nunique()  # 감독 수 (중복 제거)
num_actors = data['출연진'].apply(lambda x: len(x.split(','))).sum()  # 출연진 수 (쉼표로 구분 후 합산)
num_genres = data['장르'].apply(lambda x: len(x.split(','))).sum()  # 장르 수 (쉼표로 구분 후 합산)

In [4]:
print(f"영화 개수: {num_movies}")
print(f"배급사 수: {num_distributors}")
print(f"감독 수: {num_directors}")
print(f"출연진 수: {num_actors}")
print(f"장르 수: {num_genres}")

영화 개수: 59
배급사 수: 52
감독 수: 58
출연진 수: 275
장르 수: 68


- 함수 작성
csv 파일을 읽어 DataFrame으로 저장한 data 변수를 파라메터로 받음
배급사가 한개 이상이 경우 ', '로 구별
감독은 여러명인 경우가 없음
출연진은 한개 이상의 경우 ', '로 구별
장르는 한개 이상의 경우 '·'나 '/'를 ','로 변경하고 
구별하는게 어떤경우는 ', '이지만 장르 경우는 ','로 구별해야 하기도 함. 따라서 정규식 표현으로 구별.

In [5]:
import re

- 정규표현식 r'[,·/]|\s*,\s*'의 의미
[,·/]: ',', '·', '/' 중 하나와 일치
|: 또는
\s*,\s*: 선택적 공백과 쉼표, 그리고 다시 선택적 공백
- 정규식 검증
(참고) https://inpa.tistory.com/entry/%F0%9F%92%BB-%EC%A0%95%EA%B7%9C%EC%8B%9D-%ED%85%8C%EC%8A%A4%ED%8A%B8-%EC%82%AC%EC%9D%B4%ED%8A%B8-%F0%9F%8E%81-%EB%AA%A8%EC%9D%8C
https://regexr.com/
https://regexr.com/8ad0p
(참고) https://yenbook.tistory.com/66
https://www.regextester.com/

In [6]:
def analyze_movie_data(data):
    # 고유한 값 추출
    unique_distributors = set()
    unique_directors = set()
    unique_cast = set()
    unique_genres = set()

    for _, row in data.iterrows():
        # 배급사 분리 및 추가
        distributors = re.split(r'[,·/]|\s*,\s*', row['배급사'])
        # distributors = row['배급사'].split(', ')
        # if len(distributors) > 1:
        #     for distributor in distributors:
        #         unique_distributors.update(distributor)
        # else:
        #     unique_distributors.update(distributors)
        unique_distributors.update(distributors)
        
        # 감독 추가
        unique_directors.add(row['감독'])

        # 출연진 분리 및 추가
        cast = re.split(r'[,·/]|\s*,\s*', row['출연진'])
        #cast = row['출연진'].split(', ')
        # if len(cast) > 1:
        #     for actor in cast:
        #         unique_cast.update(actor)
        # else:
        #     unique_cast.update(cast)
        unique_cast.update(cast)
        
        # 장르 분리 및 추가
        row['장르'] = row['장르'].replace('·', ', ')
        row['장르'] = row['장르'].replace('/', ', ')
        #genres = row['장르'].split(', ')
        genres = re.split(r'[,·/]|\s*,\s*', row['장르'])
        # print("genres: ", genres)
        unique_genres.update(genres)

    # 결과 계산
    num_movies = len(data)
    num_distributors = len(unique_distributors)
    num_directors = len(unique_directors)
    num_cast = len(unique_cast)
    num_genres = len(unique_genres)
    # print("장르: ", unique_genres)
    return num_movies, num_distributors, num_directors, num_cast, num_genres, unique_genres, unique_cast

In [7]:
# 데이터 분석 실행
result = analyze_movie_data(data)
print(f"영화 개수: {result[0]}")
print(f"배급사 수: {num_distributors}, 배급사 수: {result[1]}")
print(f"감독 수: {num_directors}, 감독 수: {result[2]}")
print(f"출연진 수: {num_actors}, 출연진 수: {result[3]}")
print(f"장르 수: {num_genres}, 장르 수: {result[4]}")

영화 개수: 59
배급사 수: 52, 배급사 수: 55
감독 수: 58, 감독 수: 58
출연진 수: 275, 출연진 수: 268
장르 수: 68, 장르 수: 19


In [8]:
print(f"고유 장르: {result[5]}")
print(f"고유 출연진: {result[6]}")

고유 장르: {' 가족', '액션', '다큐멘터리', '로맨스', ' 미스터리', ' 스럴리', ' 멜로', '멜로', ' 코미디', '스릴러', '범죄', ' 로맨스', '공포', '드라마', 'SF', '기타', '코미디', ' 드라마', ' 액션'}
고유 출연진: {' 정수하', ' 이윤지', ' 홍준영', '설경구', ' 김도현', ' 박해인', ' 한태경', '김다미', ' 한승도', ' 박강섭', ' 이장유', ' 한수연', '정영주', '지일주', ' 고창석', '천우희', ' 설유진', ' 채수아', ' 이지원', ' 전석호', ' 배진아', ' 김현주', ' 강진아', ' 백지혜', ' 김모범', ' 박현숙', ' 한가은', ' 하민호', ' 박수영', ' 류경수', ' 전소니', ' 정다은', ' 이현우', ' 김무열', ' 박영훈', '차선우', '김선호', ' 김승수', ' 송보은', ' 한규원', ' 이이경', ' 노상현', '마동석', ' 김기호', ' 이경욱', ' 김민경', '박호산', '김정팔', ' 김승윤', ' 이범수', ' 정은채', ' 서벽준', ' 아오키 무네타카', ' 최성원', ' 노수산나', '이동휘', ' 인내상', ' 이설구', ' 유지애', ' 이용녀', ' 아이유', ' 강윤', ' 박유신', ' 신연서', ' 이규복', ' 강길우', '오원빈', ' 심현탁', '김경일', ' 공민정', ' 서지안', '선동혁', ' 김민', ' 정인기', ' 심소영', '허준석', '이재원', '강수연', ' 김영웅', ' 정유연', ' 이세호', '박성일', '윤박', ' 임시완', ' 김민재', '안재홍', '유연석', '진선규', ' 공명', ' 박해수', ' 곽민규', ' 이하늬', ' 박동기', ' 변중희', ' 나현준', ' 김상욱', ' 강태주', ' 김명찬', ' 김희정', ' 최동구', ' 김세환', '김재화', ' 이선균', '소이', ' 박소이', '이하늬', ' 박혜영', ' 이준혁', '김명임', '

In [9]:
# 출연진의 중복을 제거하고 고유한 출연진 수를 계산
unique_actors = set()

def add_actors(actors_string):
    if isinstance(actors_string, str):
        actors = [actor.strip() for actor in actors_string.split(',')]
        unique_actors.update(actors)

data['출연진'].apply(add_actors)
num_unique_actors = len(unique_actors)

In [10]:
num = 0
for actor in unique_actors:
  num += 1
  print (num, actor)

1 이문식
2 김민희
3 김우겸
4 설경구
5 김다미
6 오달수
7 지연
8 이이경
9 박지수
10 박미현
11 지일주
12 이설구
13 정영주
14 현빈
15 박소이
16 정병훈
17 김민재
18 천우희
19 곽민규
20 김도현
21 김소형
22 김준형
23 윤제문
24 장원혁
25 윤가현
26 고창석
27 심영은
28 김현주
29 이한주
30 이성민
31 이규복
32 차선우
33 아오키 무네타카
34 박우영
35 최우준
36 이경준
37 김선호
38 유지애
39 강진아
40 강태주
41 성유빈
42 임지섭
43 마동석
44 서벽준
45 박소감
46 전혜진
47 신의현
48 이지훈
49 박호산
50 정승길
51 정인기
52 김정팔
53 이세호
54 김세환
55 박해인
56 안미나
57 장희령
58 송보은
59 남보라
60 공민정
61 우지현
62 이동휘
63 장영준
64 김무열
65 김종수
66 박순천
67 송덕호
68 김강우
69 이미경
70 오원빈
71 김민
72 김경일
73 선동혁
74 강기영
75 박강섭
76 이재원
77 허준석
78 강수연
79 설유진
80 고규필
81 박성일
82 조수빈
83 윤박
84 김소령
85 방진원
86 안재홍
87 유연석
88 박정철
89 진선규
90 안세호
91 박혜영
92 류경수
93 신애리
94 공명
95 정서하
96 김모범
97 김재화
98 소이
99 박선호
100 이하늬
101 신연서
102 김명임
103 장동주
104 황정민
105 김희정
106 정수하
107 김정균
108 전석호
109 노지유
110 정혜인
111 인내상
112 한규원
113 이대연
114 이지원
115 김준
116 정은채
117 정수지
118 양한열
119 전소니
120 최민수
121 서지희
122 김영웅
123 배진아
124 심희섭
125 홍준영
126 설인아
127 김다현
128 정다은
129 서은채
130 박동기
131 박소진
132 한수연
133 하성국
134 박상욱
135 문지인
136 양홍주
137 최지연
138 이용녀
139 노상

** 영화 추천 방식 **
개인 취양 장르/배우 로 추천
추천 갯수 입력 (3~5개 이하 권장)